In [1]:
import cupy as cp
import numpy as np
import rasterio
from rasterio import Affine, CRS

def write_output(output, name):
    """
    Write grid to .tiff file.
    ----
    Input:
    - dataset: Can be either a rasterio dataset (for rasters) or laspy dataset (for point clouds)
    - output (Array): the output grid, a numpy grid.
    - name (String): the name of the output file.
    - transform:
      a user defined rasterio Affine object, used for the transforming the pixel coordinates
      to spatial coordinates.
    - change_nodata (Boolean): true: use a no data value of -9999, false: use the datasets no data value
    """
    output_file = name

    output = np.squeeze(output)
    # Set the nodata value: use -9999 if nodata_value is True or dataset does not have nodata.
    crs = CRS.from_epsg(28992)
    nodata_value = -9999
    # transform = Affine(0.50, 0.00, 119300.00,
    #                    0.00, -0.50, 486500.00)
    transform = Affine(0.50, 0.00, 153100.0,
                       0.00, -0.50,  471200.0)

    # output the dataset
    with rasterio.open(output_file, 'w',
                       driver='GTiff',
                       height=output.shape[0],  # Assuming output is (rows, cols)
                       width=output.shape[1],
                       count=1,
                       dtype=np.float32,
                       crs=crs,
                       nodata=nodata_value,
                       transform=transform) as dst:
        dst.write(output, 1)
    print("File written to '%s'" % output_file)

In [ ]:
def shadowingfunction_20_cupy(a, vegdem, vegdem2, azimuth, altitude, scale, amaxvalue, trunkcheck, bush, forsvf, savepath, dsm):
    # Conversion
    degrees = cp.pi / 180.0
    azimuth = azimuth * degrees
    altitude = altitude * degrees

    # Grid size
    sizex, sizey = a.shape

    # Initialize parameters
    dx = dy = dz = 0.0
    temp = cp.zeros((sizex, sizey), dtype=cp.float32)
    tempvegdem = cp.zeros((sizex, sizey), dtype=cp.float32)
    tempvegdem2 = cp.zeros((sizex, sizey), dtype=cp.float32)
    templastfabovea = cp.zeros((sizex, sizey))
    templastgabovea = cp.zeros((sizex, sizey))
    bushplant = bush > 1.0
    sh = cp.zeros((sizex, sizey), dtype=cp.float32)
    vbshvegsh = cp.zeros((sizex, sizey), dtype=cp.float32)
    vegsh = cp.array(bushplant, dtype=cp.float32)

    f = cp.array(a, dtype=cp.float32)

    # Precompute trigonometric values
    pibyfour = cp.pi / 4.0
    threetimespibyfour = 3.0 * pibyfour
    fivetimespibyfour = 5.0 * pibyfour
    seventimespibyfour = 7.0 * pibyfour
    sinazimuth = cp.sin(azimuth)
    cosazimuth = cp.cos(azimuth)
    tanazimuth = cp.tan(azimuth)
    signsinazimuth = cp.sign(sinazimuth)
    signcosazimuth = cp.sign(cosazimuth)
    dssin = cp.abs(1.0 / sinazimuth)
    dscos = cp.abs(1.0 / cosazimuth)
    tanaltitudebyscale = cp.tan(altitude) / scale

    isVert = ((pibyfour <= azimuth) & (azimuth < threetimespibyfour)) | \
             ((fivetimespibyfour <= azimuth) & (azimuth < seventimespibyfour))
    if isVert:
        ds = dssin
    else:
        ds = dscos

    max_steps = int(round(min(amaxvalue / ds, min(sizex, sizey))))

    index = cp.arange(1, max_steps + 1)
    dx = cp.where(isVert, -signcosazimuth * cp.abs(cp.round(index / tanazimuth)), -signcosazimuth * index)
    dy = cp.where(isVert, signsinazimuth * index, signsinazimuth * cp.abs(cp.round(index * tanazimuth)))
    dz = (ds * index) * tanaltitudebyscale

    preva = a - ds * tanaltitudebyscale

    index = 0

    for i in range(max_steps):

        dz = (ds * index) * tanaltitudebyscale

        tempvegdem.fill(np.nan)
        tempvegdem2.fill(np.nan)
        temp.fill(0)


        absdx = cp.abs(dx[i])
        absdy = cp.abs(dy[i])
        xc1 = int((dx[i] + absdx) / 2.0)
        xc2 = int(sizex + (dx[i] - absdx) / 2.0)
        yc1 = int((dy[i] + absdy) / 2.0)
        yc2 = int(sizey + (dy[i] - absdy) / 2.0)
        xp1 = int(-((dx[i] - absdx) / 2.0))
        xp2 = int(sizex - (dx[i] + absdx) / 2.0)
        yp1 = int(-((dy[i] - absdy) / 2.0))
        yp2 = int(sizey - (dy[i] + absdy) / 2.0)


        temp[xp1:xp2, yp1:yp2] = a[xc1:xc2, yc1:yc2] - dz[i]

        f = cp.fmax(f, temp)
        sh = cp.where(f > a, 1.0, 0.0)

        tempvegdem[xp1:xp2, yp1:yp2] = vegdem[xc1:xc2, yc1:yc2] - dz[i]
        fabovea = tempvegdem > a
        lastfabovea = tempvegdem > preva

        tempvegdem2[xp1:xp2, yp1:yp2] = vegdem2[xc1:xc2, yc1:yc2] - dz[i]
        gabovea = tempvegdem2 > a
        lastgabovea = tempvegdem2 > preva

        vegsh2 = cp.add(cp.add(cp.add(fabovea, gabovea, dtype=cp.float32), lastfabovea, dtype=cp.float32),
                        lastgabovea, dtype=cp.float32)

        vegsh2 = cp.where(vegsh2 == 4.0, 0.0, vegsh2)
        vegsh2 = cp.where(vegsh2 > 0.0, 1.0, vegsh2)

        vegsh = cp.fmax(vegsh, vegsh2)
        vegsh = cp.where(vegsh * sh > 0.0, 0.0, vegsh)
        cp.add(vbshvegsh, vegsh, out=vbshvegsh)

        index += 1.0

    sh = 1.0 - sh
    vbshvegsh[vbshvegsh > 0.0] = 1.0
    vbshvegsh -= vegsh
    vegsh = 1.0 - vegsh
    vbshvegsh = 1.0 - vbshvegsh

    shadowresult = {
        'sh': sh,
        'vegsh': vegsh,
        'vbshvegsh': vbshvegsh
    }
    return shadowresult